In [ ]:
import requests
from lxml import html

def scrape_all_elements_with_xpath(url, xpath):
    try:
        # Envoyer une requête HTTP GET
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs HTTP

        # Convertir le contenu en arbre DOM
        tree = html.fromstring(response.content)

        # Extraire tous les éléments en utilisant XPath
        elements = tree.xpath(xpath)
        if elements:
            print(f"Nombre d'éléments trouvés : {len(elements)}")
            for i, element in enumerate(elements, start=1):
                print(f"Élément {i} : {element.text_content().strip()}")
        else:
            print("Aucun élément trouvé pour le chemin donné.")

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la connexion au site : {e}")
    except Exception as e:
        print(f"Une erreur est survenue : {e}")

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    url = "https://www2.assemblee-nationale.fr/documents/liste/(type)/index-promulgations"
    xpath = "/html/body/div/div[2]/div/div/section/div/article/div/div[2]/div[2]/ul/li/p"
    scrape_all_elements_with_xpath(url, xpath)

Nombre d'éléments trouvés : 10
Élément 1 : Portant réforme du financement de l'audiovisuel public
Élément 2 : De finances de fin de gestion pour 2024
Élément 3 : Visant à sécuriser le mécanisme de purge des nullités
Élément 4 : Visant à renforcer les outils de régulation des meublés de tourisme à l'échelle locale
Élément 5 : Visant à améliorer le repérage et l’accompagnement des personnes présentant des troubles du neuro-développement et à favoriser le répit des proches aidants
Élément 6 : Visant à poursuivre l’expérimentation relative au travail à temps partagé aux fins d’employabilité
Élément 7 : Visant à reporter le renouvellement général des membres du congrès et des assemblées de province de la Nouvelle-Calédonie
Élément 8 : Autorisant la ratification de l’accord se rapportant à la convention des Nations unies sur le droit de la mer et portant sur la conservation et l’utilisation durable de la diversité biologique marine des zones ne relevant pas de la juridiction nationale
Élémen

In [ ]:
import requests
from lxml import html
from collections import Counter

def scrape_legifrance_links(url, xpath):
    try:
        # Envoyer une requête HTTP GET
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs HTTP

        # Convertir le contenu en arbre DOM
        tree = html.fromstring(response.content)

        # Extraire tous les éléments correspondant au chemin XPath
        elements = tree.xpath(xpath)
        legifrance_links = []  # Liste pour stocker les liens Legifrance

        if elements:
            print(f"Nombre d'éléments trouvés : {len(elements)}")
            for i, element in enumerate(elements, start=1):
                link_href = element.get("href", "Pas de lien")
                if "legifrance" in link_href:  # Filtrer uniquement les URLs contenant "legifrance"
                    legifrance_links.append(link_href)
                    print(f"Lien {i} : {link_href}")
        else:
            print("Aucun élément trouvé pour le chemin donné.")

        return legifrance_links

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la connexion au site : {e}")
        return []
    except Exception as e:
        print(f"Une erreur est survenue : {e}")
        return []

def analyze_keywords_in_links(links, keywords):
    keyword_counts = {}
    for link in links:
        try:
            # Envoyer une requête HTTP GET pour chaque lien
            response = requests.get(link)
            response.raise_for_status()  # Vérifie les erreurs HTTP

            # Compter les occurrences des mots clés
            page_content = response.text.lower()
            counts = Counter({keyword: page_content.count(keyword) for keyword in keywords})
            keyword_counts[link] = counts
            print(f"Analyse pour {link} : {counts}")

        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la connexion au lien {link} : {e}")

    # Associer chaque lien au mot avec le plus d'occurrences
    link_to_top_keyword = {}
    for link, counts in keyword_counts.items():
        if counts:
            max_count = max(counts.values())
            if max_count > 0:
                top_keywords = [keyword for keyword, count in counts.items() if count == max_count]
                link_to_top_keyword[link] = top_keywords

    return link_to_top_keyword

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    url = "https://www2.assemblee-nationale.fr/documents/liste/(type)/index-promulgations"
    xpath = "/html/body/div/div[2]/div/div/section/div/article/div/div[2]/div[2]/ul/li/ul/li/a"
    keywords = ["intérieur", "armées", "diplomatie", "justice", "affaires étrangères", "économie", "finances", "santé", "culture", "logement"]

    legifrance_links = scrape_legifrance_links(url, xpath)
    link_to_top_keyword = analyze_keywords_in_links(legifrance_links, keywords)

    print("Résumé des mots-clés principaux associés à chaque lien :")
    for link, top_keywords in link_to_top_keyword.items():
        print(f"{link} : {', '.join(top_keywords)}")

Nombre d'éléments trouvés : 20
Lien 2 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=MICX2426243L
Lien 4 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=BCPX2428679L
Lien 6 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=JUSX2415328L
Lien 8 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=TECX2330139L
Lien 10 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=MSAC2402474L
Lien 12 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=TEMT2401603L
Lien 14 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=INTX2426241L
Lien 16 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=EAEJ2405751L
Lien 18 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=EAEJ2333583L
Lien 20 : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=IOMX2408435L
Analyse pour http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=MICX2426243L : Counter({'finances': 3, 'diplomatie': 2, 'affaires étrangères': 1, 'économie': 1, 'cultu

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def scrape_and_summarize_links(links):
    link_summaries = {}

    # Initialiser le pipeline de résumé Hugging Face
    summarizer = pipeline("summarization")

    for link in links:
        try:
            # Envoyer une requête HTTP GET pour chaque lien
            response = requests.get(link)
            response.raise_for_status()  # Vérifie les erreurs HTTP

            # Parser le contenu HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le texte du contenu principal
            page_text = ' '.join([p.text for p in soup.find_all('p')])

            # Résumer le contenu avec Hugging Face
            if page_text.strip():
                try:
                    summary = summarizer(page_text, max_length=130, min_length=30, do_sample=False)
                    link_summaries[link] = summary[0]['summary_text'] if summary else "Résumé non disponible."
                except Exception as e:
                    link_summaries[link] = f"Erreur lors du résumé : {e}"
            else:
                link_summaries[link] = "Aucun texte trouvé sur la page."

        except requests.exceptions.RequestException as e:
            link_summaries[link] = f"Erreur lors de la connexion : {e}"

    return link_summaries

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    links = legifrance_links

    summaries = scrape_and_summarize_links(links)

    print("Résumé des contenus des liens :")
    for link, summary in summaries.items():
        print(f"\nLien : {link}\nRésumé : {summary}")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu
Token indices sequence length is longer than the specified maximum sequence length for this model (16819 > 1024). Running this sequence through the model will result in indexing errors


Résumé des contenus des liens :

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=MICX2426243L
Résumé :  L'Assemblée nationale et le Sénat have adopted la loi n° 2024-1177 . La présente loi sera exécutée comme loi de l'Etat. Fait à Paris, le 13 décembre 2024 . Participez en répondant à cette enquête, en quelques minutes!

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=BCPX2428679L
Résumé : Erreur lors du résumé : index out of range in self

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=JUSX2415328L
Résumé : Erreur lors du résumé : index out of range in self

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=TECX2330139L
Résumé : Erreur lors du résumé : index out of range in self

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=MSAC2402474L
Résumé : Erreur lors du résumé : index out of range in self

Lien : http://www.legifrance.gouv.fr/WAspad/UnTexteDeJorf?numjo=TEMT2401603L
Résumé : Erreur lors du résumé :

In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def split_text(text, max_words=500):
    """Divise le texte en segments de max_words mots."""
    words = text.split()
    for i in range(0, len(words), max_words):
        yield " ".join(words[i:i + max_words])

def extract_sentences_before_article(text):
    """Extrait les deux phrases avant 'Article 1er'."""
    sentences = text.split('. ')
    for i, sentence in enumerate(sentences):
        if "article 1 er" in sentence:
            return ". ".join(sentences[max(0, i-2):i])
    return "Pas de contenu avant 'Article 1er'."

def scrape_and_summarize_links(links):
    link_summaries = {}

    # Initialiser le pipeline de résumé Hugging Face
    summarizer = pipeline("summarization")

    for link in links:
        try:
            # Envoyer une requête HTTP GET pour chaque lien
            response = requests.get(link)
            response.raise_for_status()  # Vérifie les erreurs HTTP

            # Parser le contenu HTML
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extraire le texte du contenu principal
            page_text = ' '.join([p.text for p in soup.find_all('p')])

            # Extraire les deux phrases avant 'Article 1er'
            relevant_text = extract_sentences_before_article(page_text)

            # Résumer le contenu extrait avec Hugging Face
            if relevant_text.strip():
                try:
                    summary = summarizer(relevant_text, max_length=130, min_length=30, do_sample=False)
                    link_summaries[link] = summary[0]['summary_text'] if summary else "Résumé non disponible."
                except Exception as e:
                    link_summaries[link] = f"Erreur lors du résumé : {str(e)}"
            else:
                link_summaries[link] = "Aucun texte pertinent trouvé sur la page."

        except requests.exceptions.RequestException as e:
            link_summaries[link] = f"Erreur lors de la connexion : {str(e)}"

    return link_summaries

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    links = [
        "https://www.legifrance.gouv.fr/dossierlegislatif/JORFDOLE000050394198/?detailType=EXPOSE_MOTIFS&detailId="
    ]  # Remplacez par vos liens

    summaries = scrape_and_summarize_links(links)

    print("Résumé des contenus des liens :")
    for link, summary in summaries.items():
        print(f"\nLien : {link}\nRésumé : {summary}")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu
Your max_length is set to 130, but your input_length is only 82. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=41)


Résumé des contenus des liens :

Lien : https://www.legifrance.gouv.fr/dossierlegislatif/JORFDOLE000050394198/?detailType=EXPOSE_MOTIFS&detailId=
Résumé :  Toutefois, en raison de la dissolution de l'Assemblée nationale, ce texte est rendu caduc . Le Sénat, en tant que chambrere haute du Parlement, doit assurer la continuité du institutions et du service public .


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def scrape_law_content(url):
    """Récupérer le texte d'une loi depuis une URL."""
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    law_text = ' '.join([p.text for p in soup.find_all('p')])
    return law_text

def summarize_law(content):
    """Générer un résumé court du texte d'une loi."""
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    summary = summarizer(content, max_length=1024, min_length=40, do_sample=False)
    return summary[0]['summary_text']

In [ ]:
# Exemple d'utilisation
if __name__ == "__main__":
    # URL d'exemple - Remplacez par une URL réelle
    url = "https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000050500482"

    try:
        # Récupérer le texte de la loi
        law_content = scrape_law_content(url)
        print("Texte intégral récupéré.")

        # Résumer le texte
        law_summary = summarize_law(law_content)
        print("Résumé de la loi :")
        print(law_summary)
    except Exception as e:
        print(f"Erreur : {e}")

Texte intégral récupéré.


Device set to use cpu


Erreur : index out of range in self


In [ ]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def fetch_law_text(url):
    """Récupère le texte d'une loi depuis une URL de Légifrance."""
    try:
        response = requests.get(url)
        response.raise_for_status()  # Vérifie les erreurs HTTP

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extraire le texte des paragraphes principaux
        paragraphs = soup.find_all('p')
        law_text = ' '.join(p.text for p in paragraphs if p.text.strip())

        if not law_text:
            raise ValueError("Aucun texte pertinent trouvé sur la page.")

        return law_text
    except Exception as e:
        print(f"Erreur lors de la récupération du texte : {e}")
        return None

# def summarize_text(text):
#     """Effectue un résumé du texte donné."""
#     try:
#         summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
#         summary = summarizer(text, max_length=130, min_length=30, do_sample=False)
#         return summary[0]['summary_text']
#     except Exception as e:
#         print(f"Erreur lors du résumé : {e}")
#         return None

def summarize_text(text, max_length=130, min_length=30):
    """Effectue un résumé du texte donné avec des vérifications de longueur."""
    try:
        summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

        # Vérifier la longueur du texte en tokens
        num_tokens = len(text.split())
        if num_tokens < 20:  # Ignorer les segments très courts
            print("Segment trop court pour être résumé.")
            return None

        adjusted_max_length = min(max_length, num_tokens)  # Ajuster max_length dynamiquement
        adjusted_min_length = min(min_length, num_tokens // 2)  # Ajuster min_length dynamiquement

        # Résumer le texte
        summary = summarizer(text, max_length=adjusted_max_length, min_length=adjusted_min_length, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        print(f"Erreur lors du résumé : {e}")
        return None


def split_text(text, max_length=128):
    """Divise le texte en segments de longueur dynamique max_length."""
    words = text.split()
    adjusted_length = min(max_length, len(words) // 5)  # Divise en au moins 5 segments
    for i in range(0, len(words), adjusted_length):
        yield " ".join(words[i:i + adjusted_length])



# from transformers import LongformerTokenizer, LongformerForSequenceClassification

# def summarize_with_longformer(text):
#     tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")
#     model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096")
#     inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)
#     summary = model.generate(inputs.input_ids, max_length=130, min_length=30, do_sample=False)
#     return tokenizer.decode(summary[0], skip_special_tokens=True)


In [ ]:
if __name__ == "__main__":
    url = "https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000050500482"
    print("Récupération du texte de la loi...")
    law_text = fetch_law_text(url)

    if law_text:
        print("Texte récupéré. Résumé en cours...")

        summaries = []
        for segment in split_text(law_text):
            summary = summarize_text(segment)
            if summary:
                summaries.append(summary)

        if summaries:
            print("Résumé de la loi :")
            print(" ".join(summaries))
        else:
            print("Impossible de résumer le texte.")
    else:
        print("Impossible de récupérer le texte de la loi.")



Récupération du texte de la loi...
Texte récupéré. Résumé en cours...


Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu
Device set to use cpu


Résumé de la loi :
Au plus tard à la rentrée scolaire 2027, sont créés :1° Dans chaque circonscription académique de l'hexagone, au moins un dispositif consacré à la scolarisation en milieu ordinaire des élèves de l’enseignement primaire présentant un trouble du neuro-développement. 2° Des personnels des établissements et des services mentionnés au 2° du I of l'article L. 312-1 du code L'article L. 112-2 est complété par un alinéa ainsi rédigé. La huitième ligne du tableau du second al inéa du I de l'article. L. 165-1 est remplacée par deux lignes ainsopportable. L'article L. 112-5 du code de l'éducation, les mots : ' et qui comporte notamment' sont remplacés. Article 4: Le personnel technique reçoivent une formation spécifique concernant l'accueil et le suivi des enfants. Article 5 I.-Le 1° du I of l'article L. 241-6 du code de l'action sociale et des familles. Les mesures propres à assurer son inclusion scolaire peuvent être accordées pour l'ensemble de la durée d'un cycle pédagogiqu